In [1]:
import ctypes

def ref_count(address):
    """
        Check address reference count without creating new references to the object.
    """
    return ctypes.c_long.from_address(address).value
    

In [2]:
class Person:
    def __init__(self, name):
        self.name = name

    def __repr__(self) -> str:
        return f"<{self.__class__.__name__} name={self.name}> @ {hex(id(self))}"

    def __del__(self):
        print(f"__del__ is called for {self}")
        

In [3]:
p = Person("Alex")

In [4]:
id_p = id(p)
ref_count(id_p)


1

In [5]:
p = None  # removes reference to the object, now it's 0, garbage collector detects it, calls __del__ and then removes the object
ref_count(id_p)

__del__ is called for <Person name=Alex> @ 0x10d257f20


0

In [6]:
p = Person("Bob")
del p

__del__ is called for <Person name=Bob> @ 0x10d2709b0


In [7]:
class Person:
    def __init__(self, name):
        self.name = name

    def __repr__(self) -> str:
        return f"<{self.__class__.__name__} name={self.name}> @ {hex(id(self))}"

    def __del__(self):
        print(f"__del__ is called for {self}")

    def gen_exception(self):
        raise ValueError("Something went wrong...")


In [8]:
p = Person("Eric")
p_id = id(p)
ref_count(p_id)

1

In [9]:
try:
    p.gen_exception()
except ValueError as e:
    error = e  # creates reference to the error, which also has a __traceback__ inside, which has reference to the object!!
    print(e)


Something went wrong...


In [10]:
ref_count(p_id)

2

In [11]:
dir(error)

['__cause__',
 '__class__',
 '__context__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__suppress_context__',
 '__traceback__',
 'add_note',
 'args',
 'with_traceback']

In [12]:
for key, value in error.__traceback__.tb_frame.f_locals.copy().items():
    # there are references to the Person class inside
    if isinstance(value, Person):
        print(key, value, id(value))

p <Person name=Eric> @ 0x10d270380 4515627904


In [13]:
ref_count(p_id)

2

In [14]:
del p

In [15]:
del error

__del__ is called for <Person name=Eric> @ 0x10d270380


In [19]:
class Person:
    def __init__(self, name):
        self.name = name

    def __repr__(self) -> str:
        return f"<{self.__class__.__name__} name={self.name}> @ {hex(id(self))}"

    def __del__(self):
        print(f"__del__ is called for {self}")

        # exceptions inside __del__ are ignored!
        # they will be shown in stderr but won't stop the program
        self.gen_exception()  

    def gen_exception(self):
        raise ValueError("Something went wrong...")


In [22]:
p = Person("Mark")
del p
print("\nProgram is still running!")

Exception ignored in: <function Person.__del__ at 0x10d23a5c0>
Traceback (most recent call last):
  File "/var/folders/54/lrgwxr353tj57qmbtt2jwqdm0000gn/T/ipykernel_1678/3727189462.py", line 13, in __del__
  File "/var/folders/54/lrgwxr353tj57qmbtt2jwqdm0000gn/T/ipykernel_1678/3727189462.py", line 16, in gen_exception
ValueError: Something went wrong...


__del__ is called for <Person name=Mark> @ 0x10d271f70

Program is still running!


In [35]:
import sys


class ErrToFile:

    def __init__(self, fname):
        self._fname = fname
        self._current_stderr = sys.stderr

    def __enter__(self):
        self._file = open(self._fname, "w")
        sys.stderr = self._file  # set standard error output to point into the file

    def __exit__(self, exc_type, exc_value, exc_tb):
        sys.stderr  = self._current_stderr
        if self._file:
            self._file.close()
        return False
    
        

In [36]:
p = Person("Andrew")
with ErrToFile("err.txt"):
    del p
    print(100)
print(200)
print(300)


__del__ is called for <Person name=Andrew> @ 0x10d396f00
100
200
300


In [37]:
with open("err.txt", "r") as f:
    print(f.readlines())

['Exception ignored in: <function Person.__del__ at 0x10d23a5c0>\n', 'Traceback (most recent call last):\n', '  File "/var/folders/54/lrgwxr353tj57qmbtt2jwqdm0000gn/T/ipykernel_1678/3727189462.py", line 13, in __del__\n', '  File "/var/folders/54/lrgwxr353tj57qmbtt2jwqdm0000gn/T/ipykernel_1678/3727189462.py", line 16, in gen_exception\n', 'ValueError: Something went wrong...\n']
